# Speeding things up with Cython
[GitHub on Cython](https://github.com/cython/cython/issues/2857)

[Stack Exchange Array Buffer e.g.](https://stackoverflow.com/questions/25295159/how-to-properly-pass-a-scipy-sparse-csr-matrix-to-a-cython-function/25434532)

[Cookbook](https://ipython-books.github.io/55-accelerating-python-code-with-cython/)

Need to compile in a .pyx file then use setup and Cythonize etc.

# Original Numpy Functions

In [1]:
import numpy as np

# define default coefficients
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 

def I(z,A,I_0=I0,k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    integral = np.zeros(len(z))
    integral[1:] = np.cumsum(k*A[1:]) 
    return I_0 * np.exp( -integral - kbg*z)

def p(I,q):
    return mumax * (1.0 - qmin/q) * (I/(h + I))

def rho(q, Rd):
    return rhomax * (qmax-q)/(qmax-qmin) * ( Rd/(m + Rd) )

def next_step(z,A, Rd, Rb,dz,dt,d):
    """Calculates next step for input arrays of length zmax"""
    
    II = I(z,A)
    q = Rb[1:-1]/A[1:-1]
    pp = p(II[1:-1],q)
    rrho = rho(q,Rd[1:-1])
    
    A_next = np.zeros(len(A))
    Rb_next = np.zeros(len(Rb))
    Rd_next = np.zeros(len(Rd))
    
    A_drift = v * (A[2:]-A[:-2]) / (2*dz)
    A_diffusion = d * (A[2:]-2*A[1:-1] + A[:-2]) / (dz**2)
    Rb_drift = v * (Rb[2:]-Rb[:-2]) / (2*dz)
    Rb_diffusion = d * (Rb[2:]-2*Rb[1:-1] + Rb[:-2]) / (dz**2)
    Rd_diffusion = d * (Rd[2:]-2*Rd[1:-1] + Rd[:-2]) / (dz**2)
    
    A_next[1:-1] = A[1:-1] + dt * ( pp*A[1:-1] -lbg*A[1:-1] - A_drift + A_diffusion )
    A_next[0] = 4*d/(2*v*dz + 3*d)*A_next[1] - d/(2*v*dz + 3*d)*A_next[2] 
    A_next[-1] = (4*A_next[-2] - A_next[-3])/3
    
    Rb_next[1:-1] = Rb[1:-1] + dt * (rrho*A[1:-1] -lbg*Rb[1:-1] - Rb_drift + Rb_diffusion )
    Rb_next[0] = 4*d/(2*v*dz + 3*d)*Rb_next[1] - d/(2*v*dz + 3*d)*Rb_next[2] 
    Rb_next[-1] = (4*Rb_next[-2] - Rb_next[-3])/3
    
    Rd_next[1:-1] = Rd[1:-1] + dt*(lbg*Rb[1:-1] -rrho*A[1:-1] + Rd_diffusion)
    Rd_next[0] = (4*Rd_next[1] - Rd_next[2])/3 
    Rd_next[-1] = ( 4*d*Rd_next[-1] - d*Rd_next[-2] + v*dz*Rb_next[-1] ) / (3*d)
    
    return A_next, Rd_next, Rb_next

def get_stationary(zmax=10.0, tmax=10.0, d=1.0, I0=300.0):
   
    dz = 0.1
    dt = dz/1000 # temporary 
    Nz = int(zmax/dz)
    Nt = int(tmax/ (1000*dt) )

    z_grid = np.arange(0,zmax,dz)
    time_steps = np.arange(0,tmax,dt)

    A_0 = 100 
    Rb_0 = 2.2
    Rd_0 = 30 
    
    A = np.zeros((Nz,Nt)) # RESULTS MATRIX: rows: deeper z-values, cols: time steps forward
    A[:,0] = A_0 * np.ones(Nz) # creates homogenous initial conditions

    Rb = np.zeros((Nz,Nt))
    Rb[:,0] = Rb_0 * np.ones(Nz)

    Rd = np.zeros((Nz,Nt))
    Rd[:,0] = Rd_0 * np.ones(Nz)

    A_next,Rd_next,Rb_next = A[:,0],Rd[:,0],Rb[:,0] #initial conditions

    i = 1
    counter = 1
    for t in time_steps[:-1]:
        A_next,Rd_next,Rb_next = next_step(z_grid,A_next,Rd_next,Rb_next,dz,dt,d)
        if ( i % 1000 == 0):
            A[:,counter],Rd[:,counter],Rb[:,counter] = A_next,Rd_next,Rb_next
            counter = counter +1
        i = i+1
        
    return A,Rb,Rd, z_grid, time_steps, Nz, Nt


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

tmax = 10.0
zmax = 10.0

In [3]:
%%time
A, Rb, Rd, z_grid, time_steps, Nz, Nt = get_stationary(zmax=zmax, tmax=tmax, d=1.0, I0=300.0)

CPU times: user 8.38 s, sys: 87.5 ms, total: 8.47 s
Wall time: 8.47 s


    For tmax = zmax = 10, d= 1.0
    CPU times: user 8.16 s, sys: 99.8 ms, total: 8.26 s
    Wall time: 8.23 s

# Cython Functions

In [1]:
%load_ext cython

In [5]:
# no changes made at all
import numpy as np

# define default coefficients
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 

def I(z,A,I_0=I0,k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    integral = np.zeros(len(z))
    integral[1:] = np.cumsum(k*A[1:]) 
    return I_0 * np.exp( -integral - kbg*z)

def p(I,q):
    return mumax * (1.0 - qmin/q) * (I/(h + I))

def rho(q, Rd):
    return rhomax * (qmax-q)/(qmax-qmin) * ( Rd/(m + Rd) )

def next_step(z,A, Rd, Rb,dz,dt,d):
    """Calculates next step for input arrays of length zmax"""
    
    II = I(z,A)
    q = Rb[1:-1]/A[1:-1]
    pp = p(II[1:-1],q)
    rrho = rho(q,Rd[1:-1])
    
    A_next = np.zeros(len(A))
    Rb_next = np.zeros(len(Rb))
    Rd_next = np.zeros(len(Rd))
    
    A_drift = v * (A[2:]-A[:-2]) / (2*dz)
    A_diffusion = d * (A[2:]-2*A[1:-1] + A[:-2]) / (dz**2)
    Rb_drift = v * (Rb[2:]-Rb[:-2]) / (2*dz)
    Rb_diffusion = d * (Rb[2:]-2*Rb[1:-1] + Rb[:-2]) / (dz**2)
    Rd_diffusion = d * (Rd[2:]-2*Rd[1:-1] + Rd[:-2]) / (dz**2)
    
    A_next[1:-1] = A[1:-1] + dt * ( pp*A[1:-1] -lbg*A[1:-1] - A_drift + A_diffusion )
    A_next[0] = 4*d/(2*v*dz + 3*d)*A_next[1] - d/(2*v*dz + 3*d)*A_next[2] 
    A_next[-1] = (4*A_next[-2] - A_next[-3])/3
    
    Rb_next[1:-1] = Rb[1:-1] + dt * (rrho*A[1:-1] -lbg*Rb[1:-1] - Rb_drift + Rb_diffusion )
    Rb_next[0] = 4*d/(2*v*dz + 3*d)*Rb_next[1] - d/(2*v*dz + 3*d)*Rb_next[2] 
    Rb_next[-1] = (4*Rb_next[-2] - Rb_next[-3])/3
    
    Rd_next[1:-1] = Rd[1:-1] + dt*(lbg*Rb[1:-1] -rrho*A[1:-1] + Rd_diffusion)
    Rd_next[0] = (4*Rd_next[1] - Rd_next[2])/3 
    Rd_next[-1] = ( 4*d*Rd_next[-1] - d*Rd_next[-2] + v*dz*Rb_next[-1] ) / (3*d)
    
    return A_next, Rd_next, Rb_next

def get_stationary(zmax=10.0, tmax=10.0, d=1.0, I0=300.0):
   
    dz = 0.1
    dt = dz/1000 # temporary 
    Nz = int(zmax/dz)
    Nt = int(tmax/ (1000*dt) )

    z_grid = np.arange(0,zmax,dz)
    time_steps = np.arange(0,tmax,dt)

    A_0 = 100 
    Rb_0 = 2.2
    Rd_0 = 30 
    
    A = np.zeros((Nz,Nt)) # RESULTS MATRIX: rows: deeper z-values, cols: time steps forward
    A[:,0] = A_0 * np.ones(Nz) # creates homogenous initial conditions

    Rb = np.zeros((Nz,Nt))
    Rb[:,0] = Rb_0 * np.ones(Nz)

    Rd = np.zeros((Nz,Nt))
    Rd[:,0] = Rd_0 * np.ones(Nz)

    A_next,Rd_next,Rb_next = A[:,0],Rd[:,0],Rb[:,0] #initial conditions

    i = 1
    counter = 1
    for t in time_steps[:-1]:
        A_next,Rd_next,Rb_next = next_step(z_grid,A_next,Rd_next,Rb_next,dz,dt,d)
        if ( i % 1000 == 0):
            A[:,counter],Rd[:,counter],Rb[:,counter] = A_next,Rd_next,Rb_next
            counter = counter +1
        i = i+1
        
    return A,Rb,Rd, z_grid, time_steps, Nz, Nt



In [6]:
%%time
A, Rb, Rd, z_grid, time_steps, Nz, Nt = get_stationary(zmax=zmax, tmax=tmax, d=1.0, I0=300.0)

CPU times: user 10.6 s, sys: 268 ms, total: 10.9 s
Wall time: 11.3 s


    CPU times: user 11.6 s, sys: 196 ms, total: 11.8 s
    Wall time: 12.4 s
    
    So it's actually a little slower

## Function by Function

In [17]:
%%cython -a

from cpython cimport array
import array

cdef array.array int_array_template = array.array('i', [])

cdef array.array newarray
cdef int arrayLength = 10

newarray = array.clone(int_array_template, arrayLength, zero=True)

cdef array.array scalar_multiply(array.array x, int y, int arrayLength):
    cdef array.array result 
    result = array.clone(x, arrayLength, zero=True)
    for i in range(1,arrayLength):
        result[i] = x[i] * y
    
    return result

In [39]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp

from cpython cimport array
import array

cdef array.array float_array_template = array.array('f', [])

cdef array.array scalar_multiply(array.array x, float y, int arrayLength):
    cdef array.array result 
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y*x[i]
    
    return result

cdef array.array vector_multiply(array.array x, array.array y, int arrayLength):
    cdef array.array result 
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y[i]*x[i]
    
    return result

cdef array.array scalar_divide(array.array x, float y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y/x[i]
    return result

cdef array.array vector_divide(array.array x, array.array y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = x[i]/y[i]
    return result

cdef array.array scalar_add(array.array x, float y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y+x[i]
    return result

cdef array.array scalar_subtract(array.array x, float y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y-x[i]
    return result

cdef array.array vector_add(array.array x, array.array y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y[i]+x[i]
    return result  
    
cdef float v, lbg, mumax, rhomax, qmax, qmin, m, h, I0, kbg, A_0, Rb_0, Rd_0
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 

def I(int Nz, array.array z, array.array A, float I_0=I0, float k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    
    cdef array.array integral = array.clone(A, Nz, zero=True)
    cdef int i = 0 
    
    integral[0] = k*A[0]
    
    for i in range(Nz):
        i = i + 1
        integral[i] = k*integral[i-1]
    
    return scalar_multiply(exp( - integral -  scalar_multiply(z, kbg, Nz)),I0,Nz)

def p(int Nz,array.array I,array.array q):
    
    cdef array.array result1 = array.clone(I, Nz, zero=True)
    cdef array.array result2 = array.clone(I, Nz, zero=True)
    cdef array.array result = array.clone(I, Nz, zero=True)
    
    result1 = scalar_subtract(scalar_divide(q,qmin,Nz),1.0,Nz)
    result2 = vector_divide(I,scalar_add(I,h,Nz),Nz)
    result = vector_multiply(result1,result2,Nz)
    result = scalar_multiply(result,mumax,Nz)
    
    return result

def rho(Nz, array.array q, array.array Rd):
    
    cdef array.array result1 = array.clone(Rd, Nz, zero=True)
    cdef array.array result2 = array.clone(Rd, Nz, zero=True)
    cdef array.array result = array.clone(Rd, Nz, zero=True)
    
    result1 = vector_divide(scalar_subtract(q,qmin,Nz),scalar_subtract(q,qmax,Nz),Nz)
    result2 = vector_divide(Rd,vector_add(Rd,m,Nz),Nz)
    result = vector_multiply(result1,result2,Nz)
    result = scalar_multiply(result,rhomax,Nz)
    
    return result


## for testing

def create_arrays(int Nz):
    cdef array.array zeros, ones, twos
    zeros = array.clone(float_array_template, Nz, zero=True)
    ones = scalar_add(zeros,1,Nz)
    twos = scalar_add(zeros,2,Nz)
    return zeros, ones, twos

In [41]:
create_arrays(10)

NameError: name 'create_arrays' is not defined

In [7]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp

ctypedef np.int32_t cINT32
ctypedef np.float_t cFLOAT

cdef float v, lbg, mumax, rhomax, qmax, qmin, m, h, I0, kbg, A_0, Rb_0, Rd_0
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 

def I(np.ndarray[cFLOAT, ndim=1] z, np.ndarray[cFLOAT, ndim=1] A, float I_0=I0, float k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    
    cdef np.ndarray[cFLOAT, ndim=1] integral
    cdef int i = 0
    
    integral = np.zeros(len(z))   ## NUMPY
    
    integral[0] = k*A[1]
    
    for i in range(len(z)):
        i = i + 1
        integral[i] = k*integral[i-1]
    
    return I_0 * exp( - integral - kbg*z)

## All Functions

In [39]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp

ctypedef np.int32_t cINT32
ctypedef np.float_t cFLOAT

cdef float v, lbg, mumax, rhomax, qmax, qmin, m, h, I0, kbg, A_0, Rb_0, Rd_0
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 

def I(np.ndarray[cFLOAT, ndim=1] z, np.ndarray[cFLOAT, ndim=1] A, float I_0=I0, float k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    
    cdef np.ndarray[cFLOAT, ndim=1] integral
    cdef int i = 0
    
    integral = np.zeros(len(z))   ## NUMPY
    
    integral[0] = k*A[1]
    
    for i in range(len(z)):
        i = i + 1
        integral[i] = k*integral[i-1]
    
    return I_0 * exp( - integral - kbg*z)

def p(np.ndarray[cFLOAT, ndim=1] I,np.ndarray[cFLOAT, ndim=1] q):
    return mumax * (1.0 - qmin/q) * (I/(h + I))

def rho(np.ndarray[cFLOAT, ndim=1] q, np.ndarray[cFLOAT, ndim=1] Rd):
    return rhomax * (qmax-q)/(qmax-qmin) * ( Rd/(m + Rd) )

def next_step(np.ndarray[cFLOAT, ndim=1] z,np.ndarray[cFLOAT, ndim=1] A,
              np.ndarray[cFLOAT, ndim=1] Rd,
              np.ndarray[cFLOAT, ndim=1] Rb,
              float dz, float dt, float d):
    """Calculates next step for input arrays of length zmax"""
    
    cdef np.ndarray[cFLOAT, ndim=1] II, q, pp, rrho, A_next, Rb_next, Rd_next
    cdef np.ndarray[cFLOAT, ndim=1] A_drift, A_diffusion
    cdef np.ndarray[cFLOAT, ndim=1] Rb_drift, Rb_diffusion, Rd_diffusion
    
    II = I(z,A)
    q = Rb[1:-1]/A[1:-1]
    pp = p(II[1:-1],q)
    rrho = rho(q,Rd[1:-1])
    
    A_next = np.zeros(len(A))
    Rb_next = np.zeros(len(Rb))
    Rd_next = np.zeros(len(Rd))
    
    A_drift = v * (A[2:]-A[:-2]) / (2*dz)
    A_diffusion = d * (A[2:]-2*A[1:-1] + A[:-2]) / (dz**2)
    Rb_drift = v * (Rb[2:]-Rb[:-2]) / (2*dz)
    Rb_diffusion = d * (Rb[2:]-2*Rb[1:-1] + Rb[:-2]) / (dz**2)
    Rd_diffusion = d * (Rd[2:]-2*Rd[1:-1] + Rd[:-2]) / (dz**2)
    
    A_next[1:-1] = A[1:-1] + dt * ( pp*A[1:-1] -lbg*A[1:-1] - A_drift + A_diffusion )
    A_next[0] = 4*d/(2*v*dz + 3*d)*A_next[1] - d/(2*v*dz + 3*d)*A_next[2] 
    A_next[-1] = (4*A_next[-2] - A_next[-3])/3
    
    Rb_next[1:-1] = Rb[1:-1] + dt * (rrho*A[1:-1] -lbg*Rb[1:-1] - Rb_drift + Rb_diffusion )
    Rb_next[0] = 4*d/(2*v*dz + 3*d)*Rb_next[1] - d/(2*v*dz + 3*d)*Rb_next[2] 
    Rb_next[-1] = (4*Rb_next[-2] - Rb_next[-3])/3
    
    Rd_next[1:-1] = Rd[1:-1] + dt*(lbg*Rb[1:-1] -rrho*A[1:-1] + Rd_diffusion)
    Rd_next[0] = (4*Rd_next[1] - Rd_next[2])/3 
    Rd_next[-1] = ( 4*d*Rd_next[-1] - d*Rd_next[-2] + v*dz*Rb_next[-1] ) / (3*d)
    
    return A_next, Rd_next, Rb_next

# leave unused function here

def get_stationary(float zmax=10.0, float tmax=10.0, float d=1.0, float I0=300.0):
    
    cdef float dz, dt, Nz, Nt, A_0, Rb_0, Rd_0
    cdef np.ndarray[cFLOAT, ndim=1] z_grid, time_steps
    cdef np.ndarray[cFLOAT, ndim=2] A, Rb, Rd
    cdef int i, counter
    
    dz = 0.1
    dt = dz/1000 # temporary 
    Nz = int(zmax/dz)
    Nt = int(tmax/ (1000*dt) )

    z_grid = np.arange(0,zmax,dz)
    time_steps = np.arange(0,tmax,dt)

    A_0 = 100 
    Rb_0 = 2.2
    Rd_0 = 30 
    
    A = np.zeros((Nz,Nt)) # RESULTS MATRIX: rows: deeper z-values, cols: time steps forward
    A[:,0] = A_0 * np.ones(Nz) # creates homogenous initial conditions

    Rb = np.zeros((Nz,Nt))
    Rb[:,0] = Rb_0 * np.ones(Nz)

    Rd = np.zeros((Nz,Nt))
    Rd[:,0] = Rd_0 * np.ones(Nz)

    A_next,Rd_next,Rb_next = A[:,0],Rd[:,0],Rb[:,0] #initial conditions

    i = 1
    counter = 1
    for t in time_steps[:-1]:
        A_next,Rd_next,Rb_next = next_step(z_grid,A_next,Rd_next,Rb_next,dz,dt,d)
        if ( i % 1000 == 0):
            A[:,counter],Rd[:,counter],Rb[:,counter] = A_next,Rd_next,Rb_next
            counter = counter +1
        i = i+1
        
    return A, Rb, Rd, z_grid, time_steps, Nz, Nt


In [31]:
#%%time
get_stationary()

TypeError: 'float' object cannot be interpreted as an integer

# Visualisation Cells

In [14]:
# Visualize Shift
plt.plot(z_grid,A[:,-1])
plt.plot(z_grid,A[:,0],color = 'r')

NameError: name 'z_grid' is not defined

In [ ]:
# Heatmap
plt.imshow(A[:,:],aspect=tmax/zmax,origin="lower",extent=(0,tmax,0,zmax))
plt.xlabel('days')
plt.ylabel('depth (meters)')
plt.gca().invert_yaxis()
plt.colorbar()

In [ ]:
# 3D Visualize
#T,Z = np.meshgrid(time_steps,z_grid)
days = np.linspace(0,tmax,Nt)
T,Z = np.meshgrid(days,z_grid)

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(T,Z, A, cmap='cool')
ax.set_xlabel('time (days)')
ax.set_ylabel('z (meters)')
ax.set_zlabel('C')
ax.set_title('A - Phytoplankton')